In [2]:
import numpy as np

lut = np.load("./LUT.npy")
lut = lut[:, [0,1,3,5,6,7]]
veg_im = np.random.random(lut.shape)
print veg_im.shape

(8708, 6)


In [3]:

def lut_squarer(x):
    return (x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2)**0.5

lut_sq = np.apply_along_axis(lut_squarer, 1, lut)

In [29]:
def get_sa2(table, sq_table, image):
    sq_data = np.sum(np.multiply(image, image))**.5
    
    #sa = np.zeros(table.shape[0])

    sq_sum = np.sum(np.multiply(table, image), axis=1)

    return np.arccos(sq_sum/(sq_data*sq_table))


#%timeit get_sa2(lut, lut_sq, veg_im)

print np.sum(np.multiply(lut, veg_im), axis=1).shape
print np.apply_along_axis(lut_squarer, 1, lut).shape
print np.sum(np.multiply(veg_im, veg_im))**.5

#print get_sa2(lut, lut_sq, veg_im)

(8708,)
(8708,)
131.527203687


In [52]:
lut = np.random.random((1000, 6))
img = np.random.random((6, ))

img_sq = np.einsum('i,i->',img, img)**.5
lut_sq = np.sqrt(np.einsum('ij,ij->i',lut, lut))
sum_sq = np.einsum('ij,j->i', lut, img)

sa = i   

In [118]:
# Test affine

import numpy as np

def get_affine_mat(gdal_geot):
    affine_str = "{}, 0.00, {}; 0.0, {}, {}; 0.00, 0.00, 1.00".format(gdal_geot[1], gdal_geot[0], 
                                                                  gdal_geot[5], gdal_geot[3])

    return np.matrix(affine_str)


def transform_mat(geot, i, j):
    
    affine = get_affine_mat(geot)
    
    point = np.matrix('{}, {}, 1'.format(i, j))
    
    return tuple((affine * point.T).A1[:2])


def get_affine(geot):
    
    return np.array([geot[1], .0, geot[0], .0, geot[5], geot[3], .0, .0, 1.]).reshape((3,3))
    


def transform(geot, i, j):
    
    return tuple(np.dot(get_affine(geot), np.array([i, j, 1]))[:2])


def transform_inv(geot, x, y):
    
    return tuple(np.dot(np.linalg.inv(get_affine(geot)), np.array([x, y, 1]))[:2])


gdal_geot = (13343406.236, 463.3127165279165, 0.0, -3335851.559, 0.0, -463.3127165279167)


print transform_mat(gdal_geot, 0, 0)
print transform_inv(gdal_geot, *transform(gdal_geot, 10, 0))


(13343406.236, -3335851.5589999999)
(10.0, 0.0)


In [26]:
def get_sa3(table, sq_table, image):
    
    return np.arccos(np.einsum('ij,ij->i',table,image)/(np.einsum('ij,ij->i',table,table)*sq_table))


%timeit get_sa3(lut, lut_sq, veg_im)
print np.einsum('ij,ij->i',lut, veg_im)
print np.einsum('ij,ij->i',lut, lut)*lut_sq
print get_sa3(lut, lut_sq, veg_im)

10000 loops, best of 3: 182 µs per loop
[ 0.49827142  0.43104235  0.69107065 ...,  0.27686534  0.47638186
  0.33244603]
[ 0.13788396  0.13502469  0.12898844 ...,  0.04602438  0.0497456
  0.06209931]
[ nan  nan  nan ...,  nan  nan  nan]


/Users/pablo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in arccos
  app.launch_new_instance()


In [15]:
%timeit np.sum(np.multiply(lut, lut), axis=1)**.5

The slowest run took 6.64 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 75.9 µs per loop


In [16]:
%timeit np.sqrt(np.einsum('ij,ij->i',lut,lut))

The slowest run took 17.91 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 59.5 µs per loop


In [20]:
lut = np.load("./LUT.npy")
lut = lut[:, [0,1,3,5,6,7]]

from numba import jit

@jit
def my_func(x):
    """Average first and last element of a 1-D array"""
    return (x[0]**2 + x[1]**2 + x[2]**2 + x[3]**2 + x[4]**2 + x[5]**2)**0.5

%timeit np.apply_along_axis(my_func, 1, lut)

10 loops, best of 3: 23.6 ms per loop


In [ ]:
import os.path

hs = [27, 28, 29, 30, 31, 32]
vs = [10, 11, 12, 13]

canvas = np.zeros((2400*len(vs), 2400*len(hs)), dtype=np.float32)


for i, h in enumerate(hs):
    for j, v in enumerate(vs):
        if os.path.isfile("/Users/pablo/Desktop/marta/MCD43A4.A2010049.h{}v{}.005.npy".format(h, v)):
            tile = np.load("/Users/pablo/Desktop/marta/MCD43A4.A2010049.h{}v{}.005.npy".format(h, v))
            canvas[j*2400:(j+1)*2400, i*2400:(i+1)*2400] = tile
 
print(canvas.shape)
plt.imshow(canvas)

In [ ]:
from pyproj import Proj
from osgeo import gdal as _
import rasterio.warp

#h31v10 2400 2400
a_geot = (14455356.755667, 463.3127165275005, 0.0, -1111950.519667, 0.0, -463.3127165274999)

new_geot = (14455356.755667-(4*2400*(463.3127165275005)), 463.3127165275005, 0.0,
           -1111950.519667, 0.0, -463.3127165274999)

dest_canvas = np.zeros((2400, 2400), dtype=np.float32)
wgs_geot = (130.0, 0.0025, 0.0, -12., 0.0, -0.0025)

modis_proj = Proj('+proj=sinu +R=6371007.181 +nadgrids=@null +wktext')
wgs84_proj = Proj(**{'init': "epsg:4326"})

rasterio.warp.reproject(a, dest_canvas, src_transform=a_geot, src_crs=modis_proj,
                                dst_transform=wgs_geot, dst_crs=wgs84_proj, resampling=rasterio.warp.RESAMPLING.nearest)

plt.imshow(dest_canvas)                 

In [ ]:
modis = osr.SpatialReference()
modis.ImportFromProj4("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")
wgs84 = osr.SpatialReference()
wgs84.ImportFromEPSG(4326)

tx = osr.CoordinateTransformation(modis, wgs84)
mod_geot = (14455356.755667, 463.3127165275005, 0.0, -1111950.519667, 0.0, -463.3127165274999)
(ulx, uly, ulz) = tx.TransformPoint(mod_geot[0], mod_geot[3])
print(ulx, uly, ulz)
(ulx, uly, ulz) = tx.TransformPoint(mod_geot[0] + mod_geot[1]*2400, mod_geot[3] + mod_geot[5]*2400)
print(ulx, uly, ulz)

In [ ]:
from osgeo import gdal, osr

plt.figure(figsize=(28,56))
plt.axis('off')

mem_drv = gdal.GetDriverByName('MEM')

# Create GDALDataset
src = mem_drv.Create('', 14400, 9600, 1, gdal.GDT_Float32)

# Proj data
#src.SetGeoTransform((14455356.755667, 463.3127165275005, 0.0, -1111950.519667, 0.0, -463.3127165274999))
src.SetGeoTransform((14455356.755667-(4*2400*(463.3127165275005)), 463.3127165275005, 0.0, -1111950.519667, 0.0, -463.3127165274999))

modis = osr.SpatialReference()
modis.ImportFromProj4("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")
src.SetProjection(modis.ExportToWkt())

band = src.GetRasterBand(1)
band.WriteArray(canvas)

print(src.GetGeoTransform())
print(src.GetProjection())

plt.imshow(src.ReadAsArray(), interpolation='none')
plt.savefig('au_veg_sinus.png', bbox_inches='tight')

In [ ]:
# Create GDALDataset
dest = mem_drv.Create('', 8800, 7000, 1, gdal.GDT_Float32)

# Proj data
dest.SetGeoTransform((111.0, 0.005, 0.0, -10., 0.0, -0.005))

wgs84 = osr.SpatialReference()
wgs84.ImportFromEPSG(4326)
dest.SetProjection(wgs84.ExportToWkt())

band = dest.GetRasterBand(1)
print(dest.RasterXSize, dest.RasterYSize)
band.WriteArray(np.zeros((7000, 8800), dtype=np.float32))

print(dest.GetGeoTransform())
print(dest.GetProjection())
#plt.imshow(dest.ReadAsArray())

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.figure(figsize=(30,44))
plt.axis('off')

GRA_type = gdal.GRA_NearestNeighbour

res = gdal.ReprojectImage(src, dest, src.GetProjection(), dest.GetProjection(), GRA_type)

plt.imshow(dest.ReadAsArray(), interpolation='none')
plt.savefig('au_veg_4326.png', bbox_inches='tight')

In [ ]:
import pandas as pd

df = pd.read_csv("./LUT.txt", sep='\t', header=None, index_col=0)

df#[[1,2,3]]